---
**Project**: Cable Burial Operability  
**Author**: Alejandra L. Cameselle  
**Date**: June 2025  
**Notebook**: 02 – Extract Depth and Slope  

### Description
Extract depth and slope values for each grid cell based on bathymetry raster.

### Inputs
- AOI grid from notebook 01 (`01_grid_100m.gpkg`)
- Bathymetry raster in meters (`bathymetry_clipped.tif`)

### Processing
- Extract mean depth and slope from bathymetry raster
- Store results in grid for further analysis

### Outputs
- `02_grid_depth_slope.gpkg`: Grid with average depth and slope per cell

### Assumptions
- Bathymetry is in meters and projected in EPSG:25829
- Slope calculated from raster using Horn method

### Dependencies
- geopandas, rasterstats, rasterio, numpy
---


In [11]:
# Libraries
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
import os

In [2]:
# Load grid
grid_path = "../processed_data/01_grid_100m.gpkg"
grid = gpd.read_file(grid_path, layer="01_grid_100m")

In [3]:
# Filter grid to inside AOI only
grid = grid[grid["inside_aoi"]].copy()
grid.reset_index(drop=True, inplace=True)

In [4]:
# Define input raster
bathy_path = "../inputs/bathymetry_clipped.tif"

In [5]:
# Compute slope raster from bathymetry
with rasterio.open(bathy_path) as src:
    bathy_array = src.read(1)
    transform = src.transform
    pixel_size = transform[0]
    bathy_crs = src.crs
    bathy_profile = src.profile

    # Compute slope in degrees
    dy, dx = np.gradient(bathy_array, pixel_size)
    slope = np.sqrt(dx**2 + dy**2)
    slope_deg = np.degrees(np.arctan(slope))

In [6]:
# Save slope raster
slope_raster_path = "../processed_data/02_slope.tif"
with rasterio.open(
    slope_raster_path,
    "w",
    driver="GTiff",
    height=slope_deg.shape[0],
    width=slope_deg.shape[1],
    count=1,
    dtype="float32",
    crs=bathy_crs,
    transform=transform,
) as dst:
    dst.write(slope_deg.astype("float32"), 1)

print("Slope raster saved:", slope_raster_path)

Slope raster saved: ../processed_data/02_slope.tif


In [7]:
# Compute mean depth per grid cell
print("Calculating mean depth...")
depth_stats = zonal_stats(
    vectors=grid["geometry"],
    raster=bathy_path,
    stats=["mean"],
    nodata=-9999
)
grid["depth_mean"] = [s["mean"] if s["mean"] is not None else np.nan for s in depth_stats]

Calculating mean depth...


In [8]:
# Compute mean slope per grid cell
print("Calculating mean slope...")
slope_stats = zonal_stats(
    vectors=grid["geometry"],
    raster=slope_raster_path,
    stats=["mean"],
    nodata=-9999
)
grid["slope_mean_deg"] = [s["mean"] if s["mean"] is not None else np.nan for s in slope_stats]

Calculating mean slope...


In [9]:
# Check for missing values
missing_depth = grid["depth_mean"].isna().sum()
missing_slope = grid["slope_mean_deg"].isna().sum()
print(f"Missing depth values: {missing_depth}, Missing slope values: {missing_slope}")

Missing depth values: 180, Missing slope values: 0


In [12]:
# Export final grid with depth and slope
os.makedirs("../processed_data", exist_ok=True)
fname = f"../processed_data/02_grid_depth_slope.gpkg"
layer = f"02_grid_depth_slope"
grid.to_file(fname, layer=layer, driver="GPKG")

print(f"Grid exported: {fname}")
print(f"Total AOI cells processed: {len(grid)}")

Grid exported: ../processed_data/02_grid_depth_slope.gpkg
Total AOI cells processed: 540242
